In [136]:
import json

num_of_json = 50000
file = 'yelp_dataset/yelp_academic_dataset_review.json'
parsed_file = 'yelp_review_small.json'
list_of_reviews_rate = []
with open(file) as f:
    with open(parsed_file, 'w') as outf:
        for i, line in enumerate(f):
            pl = json.loads(line)
            json.dump({"text": pl["text"], "label":pl["stars"]}, outf)
            outf.write('\n')
            if i==num_of_json:
                break




In [137]:
import torch
from torchtext import data

#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.long,batch_first=True)
datafield = {"text": ("text",TEXT),
                     "label": ("label",LABEL)
             }

#loading custom dataset
training_data = data.TabularDataset(path = 'yelp_review_small.json',format = 'json',fields = datafield)

#print preprocessed text
print(len(training_data.examples))

50001


In [138]:
import random
train_data, test_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))


In [139]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))

#Word dictionary
print(TEXT.vocab.stoi)

Size of TEXT vocabulary: 25792
Size of LABEL vocabulary: 5
[('.', 222758), ('the', 151650), ('and', 127048), (',', 123323), ('I', 107846), ('a', 90596), ('to', 87905), ('was', 68232), ('of', 51687), (' ', 49308)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x126ef4390>>, {'<unk>': 0, '<pad>': 1, '.': 2, 'the': 3, 'and': 4, ',': 5, 'I': 6, 'a': 7, 'to': 8, 'was': 9, 'of': 10, ' ': 11, '!': 12, 'is': 13, 'it': 14, 'for': 15, 'in': 16, 'The': 17, 'that': 18, 'with': 19, 'my': 20, '\n\n': 21, 'you': 22, 'but': 23, 'on': 24, "n't": 25, 'have': 26, 'this': 27, 'they': 28, 'had': 29, "'s": 30, 'were': 31, 'not': 32, 'we': 33, 'are': 34, 'at': 35, 'place': 36, '-': 37, 'so': 38, 'food': 39, 'good': 40, 'be': 41, 'me': 42, 'as': 43, ')': 44, 'very': 45, '(': 46, 'here': 47, 'out': 48, 'We': 49, 'great': 50, 'there': 51, 'all': 52, '"': 53, 'like': 54, 'time': 55, '\n': 56, 'service': 57, 'It': 58, 'get': 59, 'do': 60, 'our': 61, 'just': 62, 'would': 63

In [140]:
#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True)


In [141]:
import torch.nn as nn

class RNN_setiment(nn.Module):
    
    ## For each element in the input sequence, each layer computes the following function:
    
    ## h_t = ReLU(W_ih*x_t+b_ih + W_hh*h_(t-1)+b_hh)
    
    ## where h_t is the hidden state at time t, x_t is the input at time t, and h_(t-1)
    ## is the hidden state of the previous layer at time t-1 or the initial hidden state at time 0
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers):

        #Constructor
        super().__init__()          

        #embedding layer
        # A simple lookup table that stores embeddings of a fixed dictionary and size.
        # This module is often used to store word embeddings and retrieve them using indices.
        # The input to the module is a list of indices, and the output is the corresponding word embeddings.
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #rnn layer
        self.rnn = nn.RNN(embedding_dim, 
                       hidden_dim, 
                       num_layers=n_layers,
                       batch_first=True)

        # linear layer towards output
        self.linear = nn.Linear(hidden_dim, output_dim)

        # activation function
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, text, text_length):

        embedded = self.embedding(text)

        ## input can be of size T x B x * where T is the length of the longest sequence (equal to lengths[0]),
        # B is the batch size, and * is any number of dimensions (including 0).
        # If batch_first is True, B x T x * input is expected.
        packed_embedding = nn.utils.rnn.pack_padded_sequence(embedded, text_length, batch_first=True)
        packed_output, hidden = self.rnn(packed_embedding)
        linear_outputs = self.linear(hidden)
        # print('linear_outputs')
        # print(linear_outputs.shape)
        # print(linear_outputs)
        output = self.softmax(linear_outputs)
        # print('output tensor:')
        # print(output.shape)
        # print(output.view)
        # print(output)
        return output

In [142]:

# hyper-parameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 5
num_layers = 1


# instantiate the Rnn sentiment classification model for Yelp
model = RNN_setiment(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers)


In [143]:
#architecture
print(model)

#No. of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

RNN_setiment(
  (embedding): Embedding(25792, 100)
  (rnn): RNN(100, 32, batch_first=True)
  (linear): Linear(in_features=32, out_features=5, bias=True)
  (softmax): LogSoftmax(dim=2)
)
The model has 2,583,653 trainable parameters
torch.Size([25792, 100])


In [144]:
import torch.optim as opt

criterion = nn.NLLLoss()
optimizer = opt.Adam(model.parameters())


def rate_from_prediction(prediction):
    top_n, top_i = prediction.topk(1)
    rate_i = top_i[0].item()
    return rate_i


def train(model, iterator, optimizer, criterion):

    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # set the model in training phase
    model.train()

    for batch in iterator:

        # resets the gradients after every batch
        optimizer.zero_grad()

        # retrieve text and no. of words
        text, text_lengths = batch.text

        # convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()
        # print('prediction tensor:')
        # print(predictions)
        # print(predictions.shape)
        # print(predictions.view)
        # print('batch.label tensor:')
        # print(batch.label)
        # print(batch.label.shape)
        # print(batch.label.view)

        # compute the loss
        loss = criterion(predictions, batch.label)

        # compute the accuracy
        guess = rate_from_prediction(predictions)
        correct = (guess == batch.label).float()
        acc = correct.sum() / len(correct)

        #backpropage the loss and compute the gradients
        loss.backward()

        #update the weights
        optimizer.step()

        #loss and accuracy
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [145]:

def evaluate(model, iterator, criterion):

    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()

    # deactivates autograd
    with torch.no_grad():

        for batch in iterator:

            #retrieve text and no. of words
            text, text_lengths = batch.text

            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()

            #compute loss and accuracy
            loss = criterion(predictions, batch.label)

            guess = rate_from_prediction(predictions)
            correct = (guess == batch.label).float()
            acc = correct.sum() / len(correct)

            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [146]:

N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)

    #evaluate the model
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 1.324 | Train Acc: 39.65%
	 Val. Loss: 1.366 |  Val. Acc: 42.28%
	Train Loss: 1.194 | Train Acc: 36.26%
	 Val. Loss: 1.166 |  Val. Acc: 35.45%
	Train Loss: 1.218 | Train Acc: 37.33%
	 Val. Loss: 1.267 |  Val. Acc: 38.63%
	Train Loss: 1.076 | Train Acc: 34.12%
	 Val. Loss: 1.126 |  Val. Acc: 31.21%
	Train Loss: 1.109 | Train Acc: 35.52%
	 Val. Loss: 1.139 |  Val. Acc: 33.96%
